In [2]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [3]:
conn = http.client.HTTPSConnection("api.covid19api.com") #COVID 19 API
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [4]:
covid1=json.loads(data)

In [5]:
df= pd.DataFrame(covid1['Countries'])
df

,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,51dfeba7-e39e-434c-bbca-0b521a01b434,Afghanistan,AF,afghanistan,0,141489,0,6213,0,82586,2021-07-20T22:22:10.094Z,{}
1,b8bad3fd-9b96-4425-8869-231004a7da0f,Albania,AL,albania,0,132697,0,2456,0,130081,2021-07-20T22:22:10.094Z,{}
2,fd13a99d-c5de-463d-a4e3-758243f8fab9,Algeria,DZ,algeria,0,154486,0,3956,0,106337,2021-07-20T22:22:10.094Z,{}
3,d7f19087-bbe6-48c4-947b-f1f97405a0c6,Andorra,AD,andorra,0,14359,0,127,0,13897,2021-07-20T22:22:10.094Z,{}
4,4f27e56e-7898-49de-8219-dd8a528b6f1e,Angola,AO,angola,0,40906,0,969,0,34790,2021-07-20T22:22:10.094Z,{}
...,...,...,...,...,...,...,...,...,...,...,...,...
187,45629dfe-ff49-4628-b016-f946e8cea1db,Venezuela (Bolivarian Republic),VE,venezuela,0,293866,0,3392,0,276952,2021-07-20T22:22:10.094Z,{}
188,7ac35478-9a15-4e32-b626-ac9d46f4b001,Viet Nam,VN,vietnam,0,60180,0,334,0,11047,2021-07-20T22:22:10.094Z,{}
189,68d6788f-c56e-4683-b9fb-ef055098d555,Yemen,YE,yemen,0,6987,0,1370,0,4162,2021-07-20T22:22:10.094Z,{}
190,c50b72b8-47a6-46f5-b17b-7edfb5d283b5,Zambia,ZM,zambia,0,186279,0,3113,0,173320,2021-07-20T22:22:10.094Z,{}


In [6]:
#drop unnecessary columns
covid2 = df.drop(columns =['CountryCode','Slug','Date','Premium'],axis=1) 

In [7]:
m = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
m

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='COVID-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [11]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [12]:
covid_final= pd.merge(covid2,coordinates,on='Country')

In [13]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)

In [14]:
covid_final.apply(plotDot, axis = 1)

m.fit_bounds(m.get_bounds())

m

In [15]:
m1 = folium.Map(tiles='StamenToner', min_zoom=2)

m1

In [16]:
deaths=covid_final['TotalDeaths'].astype(float)

In [17]:
lat=covid_final['latitude'].astype(float)

In [18]:
lon=covid_final['longitude'].astype(float)

In [19]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

In [20]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)

covid_final.apply(plotDot, axis = 1)

m1.fit_bounds(m1.get_bounds())

m1